In [18]:
from PIL import Image
import numpy as np
from numpy import asarray, expand_dims
from os import listdir
from os.path import isdir

In [19]:
#function to load face
def load_face(filename):
    #load file image
    image = Image.open(filename)
    #convert to RGB
    image = image.convert("RGB")
    
    return asarray(image)

In [20]:
#function to load faces from each file
def load_faces(class_dir):
    faces = list()
    #iterate in folder
    for filename in listdir(class_dir):
        path = class_dir + filename
        try:
            faces.append(load_face(path))
        except:
            print("Erro ao carregar a imagem: " + path)
    return faces

In [21]:
#function to load all dataset of each class
def load_all_faces(source_dir):
    X, y = list(), list()
    #iterate by classes
    for class_dir in listdir(source_dir):
        path = source_dir + class_dir + '/'
        
        if not isdir(path):
            pass
        #load faces of the class_dir
        faces = load_faces(path)
        #fill list with class_dir as the label of the face
        labels = [class_dir for _ in range(len(faces))]
        
        #result
        print("Carregado %d faces da classe: %s" %(len(faces), class_dir))
        
        X.extend(faces)
        y.extend(labels)
    return asarray(X),asarray(y)

In [22]:
## Loading all images
trainX, trainy = load_all_faces('./Dataset/Treino/')
valX, valy = load_all_faces('./Dataset/Validacao/')

Carregado 46 faces da classe: Dessa
Carregado 47 faces da classe: Diego
Carregado 47 faces da classe: Gui
Carregado 7 faces da classe: Dessa
Carregado 7 faces da classe: Diego
Carregado 7 faces da classe: Gui


In [23]:
from tensorflow.keras.models import load_model

In [24]:
#Load facenet
model = load_model('facenet_keras.h5')
model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

In [25]:
#Generate Embeddings
def get_embedding(model, face_pixels):
    #Standardization (reduzir a magnitude de dados)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean)/std
    
    #expand dims -> (1,160,160)
    samples = expand_dims(face_pixels, axis=0)
    
    #predict to generate the embedding
    #[[1,2,...,128],[1,2,...,128]...]
    yhat = model.predict(samples)
    
    return yhat[0]

In [26]:
newTrainX = list()
newValX = list()

In [27]:
#dados treinamento
for face_pixels in trainX:
    newTrainX.append(get_embedding(model, face_pixels))
#dados validacao
for face_pixels in valX:
    newValX.append(get_embedding(model, face_pixels))
    

In [28]:
# embeddings to array list
newTrainX = asarray(newTrainX)
newValX = asarray(newValX)
print(newTrainX.shape,newValX.shape)

(140, 128) (21, 128)


In [29]:
import pandas as pd
#gera dataframe with each embedding
df_train = pd.DataFrame(data=newTrainX)
df_val = pd.DataFrame(data=newValX)

In [30]:
#add the class label to each image
df_train['target'] = trainy
df_val['target'] = valy

In [31]:
df_train.head(5)

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,target
0,-1.647590,-2.050291,2.192740,-0.310849,-0.309029,0.080395,0.863480,1.833071,1.469538,-2.226735,...,-0.453191,1.357549,-0.688587,0.387791,-0.474701,-1.109110,0.298009,-0.699501,0.078977,Dessa
1,-0.861166,-1.164513,1.859923,-1.720224,0.208900,-0.391139,1.063609,2.063228,0.459863,-2.164250,...,-1.101685,1.791857,-0.318591,1.416495,-1.061486,-0.713410,-1.610592,-0.653775,-0.210192,Dessa
2,0.195232,-1.181850,2.093904,-2.218027,-0.438787,-0.280497,-1.047070,2.230372,0.549884,-1.247451,...,-0.885988,1.448878,-0.477330,0.779704,-1.425694,0.066216,-1.969161,-1.082669,0.812740,Dessa
3,-1.604782,-2.422320,1.517443,-1.383071,0.588017,-0.112119,0.626909,2.273672,0.968901,-1.612194,...,-1.473051,1.782758,0.337711,0.581356,-0.447030,-0.064406,-0.662852,-0.813822,-0.058389,Dessa
4,-1.100709,-1.527732,1.620708,-1.118424,-0.075956,-0.231456,1.632379,1.286139,0.501847,-1.417777,...,-0.316608,0.618175,-0.029974,1.216974,-0.384685,-0.779763,-0.021223,-1.605424,-0.730062,Dessa


In [32]:
df_val.head(5)

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,target
0,-1.523674,-3.057675,2.235133,-0.341293,0.063947,-0.893874,-0.114136,1.836419,1.097465,-0.637791,...,-1.260578,1.376232,-0.643364,-0.448841,0.396284,-0.904877,-0.035660,-0.535704,0.196821,Dessa
1,-0.495632,-1.320794,2.360189,-0.847709,-0.204602,-0.841331,1.006684,1.754102,1.165373,-0.595010,...,-1.281500,1.451785,-0.256364,0.381176,-1.250163,-1.245942,-0.110298,-0.262677,-0.047439,Dessa
2,-2.179586,-2.875247,1.996602,-0.845462,0.786506,-0.807546,1.108842,1.902001,0.851351,-0.701958,...,-2.290497,1.660553,0.071010,-0.961497,-0.422987,-1.232594,-0.483871,-1.079012,-0.511883,Dessa
3,-0.808715,-1.780204,0.966140,-0.932362,0.719983,0.031120,0.716178,1.465917,1.569647,-0.558551,...,-1.817197,2.587652,0.108394,-0.914538,0.068641,-0.396750,-0.759598,-0.806594,0.179841,Dessa
4,-1.365400,-2.052475,2.176363,-0.699374,0.687984,0.476967,0.725367,1.700635,1.214217,-1.893633,...,-1.067907,1.063823,-0.322278,0.603204,-0.668626,0.130464,0.175355,-0.407748,-0.088077,Dessa


In [16]:
df_train.to_csv('faces_train_embeddings.csv')
df_val.to_csv('faces_validation_embeddings.csv')